In [1]:
import sys
sys.path.append("../")
import io

from document_search.ocr import DocumentReader, EntityProcessor


In [2]:
documents = [
    "../data/SP-51.13330.2011.pdf",
    "../data/SP-60.13330.2020.pdf"
]


In [3]:
doc_reader = DocumentReader()

raw_documents = []
processed_docs = []
for document_path in documents:
    with open(document_path, 'rb') as f:
        raw_documents.append(io.BytesIO(f.read()))

    processed_doc, errors = doc_reader.read(document_path)
    print(document_path, errors)
    processed_doc.entities = EntityProcessor.merge_text_entities(processed_doc.entities)
    processed_doc.entities = EntityProcessor.filter_short_entities(processed_doc.entities)
    processed_docs.append(processed_doc)


../data/SP-51.13330.2011.pdf []
../data/SP-60.13330.2020.pdf []


In [4]:
from document_search.storages import DocumentStorageE5
from document_search.search import TextEntityEmbedderE5

embedder = TextEntityEmbedderE5()
storage = DocumentStorageE5(embedder)


/home/atom/igor/hacks/document-search/notebooks/../document_search/search/embedders.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  self.embedder = HuggingFaceEmbeddings(
/home/atom/anaconda3/envs/doc-search2/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [5]:
for doc, raw_doc in zip(processed_docs, raw_documents):
    storage.add_document(doc, raw_doc, pbar=True)


55it [00:48,  1.13it/s]                        
120it [00:48,  2.46it/s]                         


In [31]:
storage.get_relevant_entities("постоянный шум", k=3, document_ids=['2a39bce37829487caaf5cc59df4afa13'])


[(TextDocEntity(position=EntityPosition(document_name='SP-60.13330.2020.pdf', page_number=110, document_id='2a39bce37829487caaf5cc59df4afa13'), text='Допустимый эквивалентный уровень шума, дБА'),
  0.85343045),
 (TextDocEntity(position=EntityPosition(document_name='SP-60.13330.2020.pdf', page_number=110, document_id='2a39bce37829487caaf5cc59df4afa13'), text='Число струй'),
  0.846436),
 (TextDocEntity(position=EntityPosition(document_name='SP-60.13330.2020.pdf', page_number=8, document_id='2a39bce37829487caaf5cc59df4afa13'), text='Примечание - Для систем аварийной вентиляции при работе или опробовании в помещениях, где установлено это оборудование, допускается согласно ГОСТ 12.1.003 источник шума не более 110 дБА постоянной звуковой мощности, а импульсный шум - не более 125 дБА звуковой мощности;'),
  0.8450053)]

In [6]:
from document_search.rag.retrievers import YandexGPTRetriever


In [7]:
rag_retriever = YandexGPTRetriever(embedder=embedder, storage=storage)


/home/atom/igor/hacks/document-search/notebooks/../document_search/rag/retrievers.py:26: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  self.llm_chain = LLMChain(llm=self.llm, prompt=self.prompt)


In [9]:
query = "какие трубопроводы не допускается прокладывать через помещения вентиляционного оборудования?"


In [9]:
answer = rag_retriever.retrieve_answer(query, k=1, context_length=0)
print(answer)


/home/atom/igor/hacks/document-search/notebooks/../document_search/rag/retrievers.py:41: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  answer: str = self.llm_chain.run(


Ответ:

Через помещение для вентиляционного оборудования не допускается прокладывать трубопроводы с легковоспламеняющимися и горючими жидкостями и газами, а также канализационные трубопроводы с прочистками и ревизиями (кроме трубопроводов ливневой канализации и водоотведения из вышележащих помещений для вентиляционного оборудования).

В приведённом контексте нет информации о том, какие ещё трубопроводы нельзя прокладывать через помещения вентиляционного оборудования.

Для точного ответа на вопрос пользователя не хватает данных о типе вентиляционного оборудования, характеристиках помещения, в котором оно расположено, и других особенностях системы вентиляции.


In [32]:
data, answer = rag_retriever.retrieve_answer_detailed(
    query, 
    k=5, rag_k=1, context_length=0
)


In [33]:
print(answer)


**Ответ:**

Согласно предоставленным отрывкам из документов, трубопроводы с легковоспламеняющимися и горючими жидкостями и газами, а также канализационные трубопроводы с прочистками и ревизиями (кроме трубопроводов ливневой канализации и водоотведения из вышележащих помещений для вентиляционного оборудования, в том числе от вентиляционного оборудования) не допускается прокладывать через помещения для вентиляционного оборудования.

Однако предоставленная информация может быть неполной, поскольку в ней нет прямого указания на то, что другие виды трубопроводов можно прокладывать через указанные помещения. 

Для получения более точного ответа рекомендуется обратиться к другим нормативным документам или проконсультироваться со специалистами по пожарной безопасности.


In [34]:
data


[(TextDocEntity(position=EntityPosition(document_name='SP-60.13330.2020.pdf', page_number=66, document_id='2a39bce37829487caaf5cc59df4afa13'), text='10.6  Через  помещение  для  вентиляционного  оборудования  не  допускается  прокладывать трубопроводы:'),
  0.91254383),
 (TextDocEntity(position=EntityPosition(document_name='SP-60.13330.2020.pdf', page_number=77, document_id='2a39bce37829487caaf5cc59df4afa13'), text='Не  допускается  прокладка  магистральных  и  разводящих  трубопроводов  систем  отопления  и внутреннего теплоснабжения через помещения жилых квартир, а также установка в них арматуры и спускных устройств общедомовых систем.'),
  0.8962804),
 (TextDocEntity(position=EntityPosition(document_name='SP-60.13330.2020.pdf', page_number=50, document_id='2a39bce37829487caaf5cc59df4afa13'), text='7.10.14  Оборудование  вытяжных  систем  производственных,  административно-бытовых  и общественных  зданий,  удаляющих  воздух  с  резким  или  неприятным  запахом  (из  общественных убор